In [2]:
import os
from typing import Optional
import nest_asyncio
import asyncio

from langchain_core.prompts import ChatPromptTemplate
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from google.adk.models.lite_llm import LiteLlm


model = LiteLlm(
    model='deepseek/deepseek-chat',
    api_key=os.getenv('DEEPSEEK_SK')
)

# 初始化大模型
try:
    llm: Optional[ChatOpenAI] = ChatOpenAI(base_url='https://api.deepseek.com', api_key=os.getenv('DEEPSEEK_SK'), model='deepseek-chat')
    
    print('='*10 + '大模型初始化成功' + '='*10)
    
except Exception as e:
    print(f'初始化大模型失败: {e}')
    llm = None
    
    
@langchain_tool
def search_information(query: str) -> str:
    """
        根据主题提供事实信息
    """    
    print(f'f\n--- ::tools: 工具调用：search_information, 查询：{query} ---')
    
    simulated_results = {
        "weather in london": "伦敦当前天气多云，气温 15°C。",
        "capital of france": "法国的首都是巴黎。",
        "population of earth": "地球人口约 80 亿。",
        "tallest mountain": "珠穆朗玛峰是海最高的山峰。",
        "default": f"模拟搜索 '{query}'：未找到具体信息，但该主题很有趣。"
    }
    
    result = simulated_results.get(query.lower(), simulated_results['default'])
    print('='*10 + '工具结果' + '='*10)
    print(result)
    return result

tools = [search_information]

# 创建工具调用agent
if llm:
    agent_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个乐于助人的助手。"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])
    
    agent = create_tool_calling_agent(llm=llm, prompt=agent_prompt, tools=tools)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
async def run_agent_with_tool(query: str):
    """用 Agent 执行查询并打印最终回复。"""
    print(f"\n‑‑‑ :emoji: Agent 运行查询：'{query}' ‑‑‑")
    try:
        response = await agent_executor.ainvoke({'input': query})
        print('='*10 + 'AI最终回复' + '='*10)
        print(response['output'])
    except Exception as e:
        print(e)
        
async def main():
    """并发多个agent查询"""
    tasks = [
        run_agent_with_tool('法国的首都是什么？'),
        run_agent_with_tool('伦敦天气如何？'),
        run_agent_with_tool('说说狗的相关信息。'),
    ]
    await asyncio.gather(*tasks)
    
nest_asyncio.apply()
asyncio.run(main())
    

ImportError: cannot import name 'is_data_content_block' from 'langchain_core.messages' (/Users/adan/code/egg/agent-demo/.venv/lib/python3.11/site-packages/langchain_core/messages/__init__.py)